<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/RAG_Llama2_Pinecone_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Llama 2+ Pinecone + LangChain**

##**Install All the Required Pakages**

In [1]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install llama-cpp-python
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.3/270.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 735.4 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py

In [2]:
!pip -q install bitsandbytes accelerate xformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00


In [3]:
!pip -q install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00


#**Import All the Required Libraries**

In [4]:
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os, sys

In [5]:
from huggingface_hub import notebook_login
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

In [6]:
from langchain import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

#**Load the Data**

In [7]:
# grab TIVDAK drug prescribing info from web
!mkdir -p DATA/TIVDAK;curl https://docs.seagen.com/Tivdak_Full_Ltr_Master.pdf -o DATA/TIVDAK/tivdak.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  452k  100  452k    0     0  1768k      0 --:--:-- --:--:-- --:--:-- 1775k


In [8]:
# save here
filepath = '/content/DATA/TIVDAK/tivdak.pdf'
loader = PyPDFLoader(filepath)
document=loader.load()

In [9]:
#!gdown "https://drive.google.com/uc?id=15hUEJQViQDxu_fnJeO_Og1hGqykCmJut&confirm=t"

In [10]:
#loader = OnlinePDFLoader("https://wolfpaulus.com/wp-content/uploads/2017/05/field-guide-to-data-science.pdf")
#loader = PyPDFLoader("/content/yolov7paper.pdf")
#loader = PyPDFLoader("/content/The-Field-Guide-to-Data-Science.pdf")

In [11]:
#data = loader.load()

In [12]:
len(document)

22

#**Split the Text into Chunks**

In [13]:
#text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

In [14]:
document_splitter=CharacterTextSplitter(separator='\n', chunk_size=1000, chunk_overlap=100)

In [26]:
docs=document_splitter.split_documents(document)

In [27]:
len(docs)

71

In [28]:
docs[0]

Document(page_content='HIGHLIGHTS OF PRESCRIBING INFORMATION   \nThese highlights do not include all the information needed to use \nTIVDAK safely and effectively. See full prescribing information for \nTIVDAK.  \nTIVDAK® (tisotumab vedotin -tftv) for injection, for intravenous use  \nInitial U.S. Approval: 2021  \n \nWARNING: OCULAR TOXICITY  \nSee full prescribing information for complete boxed warning.  \n \n• TIVDAK caused changes in the corneal epithelium and \nconjunctiva resulting in changes in vision, including severe vision \nloss, and corneal ulceration. ( 5.1) \n• Conduct an ophthalmic exam at baseline, prior to each dose, and \nas clinically indicated. ( 2.2, 5.1) \n• Adhere to premedication and required eye care before, during, \nand after infusion. ( 2.2) \n• Withhold TIVDAK until improvement and resume, reduce the \ndose, or permanently discontinue, based on severity. ( 2.3, 5.1) \n --------------------------   RECENT MAJOR CHANGES  --------------------------', metadata=

#**Downlaod the Embeddings**

In [15]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

#**Initializing Pinecone and Create Index**

In [33]:
# initialize pinecone
pinecone.init(
    api_key='34357661-cd66-4c00-8142-c49fc49a1b79',  # find at app.pinecone.io
    environment='us-west1-gcp-free'  # next to api key in console
)

index_name = "langchain-llama-rag" # put in the name of your pinecone index here

In [35]:
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='cosine',
        dimension=384,  # 1536 dim of text-embedding-ada-002 - 384
    )

In [ ]:
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '34357661-cd66-4c00-8142-c49fc49a1b79')
# PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', 'us-west1-gcp-free')

# pinecone.init(
#     api_key=PINECONE_API_KEY,  # find at app.pinecone.io
#     environment=PINECONE_API_ENV  # next to api key in console
# )

In [46]:
# pc_index = pinecone.Index(index_name)
# pc_index.describe_index_stats()

In [47]:
#pc_index.upsert(docs)

In [48]:
#for batch in docs.iter_documents(batch_size=10):
#    pc_index.upsert(docs)

#**Create Embeddings for Each of the Text Chunk and store in the Pinecone Index**

In [39]:
docsearch=Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name=index_name)

# Next time, because we already made this index, we can load it like this


In [38]:
#docsearch = Pinecone.from_existing_index(index_name, embeddings)


#**Test Similarity Search**

In [43]:
#query="What are examples of good data science teams?"
query="what is tivdak used for?"

In [44]:
texts=docsearch.similarity_search(query)

In [45]:
texts

[Document(page_content='General information about the safe and effective use of TIVDAK.  \nMedicines are sometimes prescribed for purposes other than those listed in a Medication Guide. If you would like more \ninformation about TIVDAK, talk with your healthcare provider. You can ask your pharmacist or healthcare provider for \ninformation about TIVDAK that is written for healthcare professionals.  \nWhat are the ingredients in TIVDAK?  \nActive ingredient:  tisotumab ve dotin -tftv \nInactive ingredients:  d-mannitol, l -histidine, l -histidine monohydrochloride, and sucrose.  \nManufactured by: Seagen Inc., Bothell, WA 98021, 1 -855-4SEAGEN  \nMarketed by: Seagen Inc., Bothell, WA 98021 and Genmab US, Inc., Plainsboro, NJ 08536  \nU.S. License  2257  \nTIVDAK® is a trademark owned by Seagen Inc.  \n©202 3 Seagen Inc. and Genmab US, Inc.  \nMG-761208 -v03 \nFor more information, go to www.tivdak.com  or call 1 -855-4SEAGEN', metadata={}),
 Document(page_content='General information ab

#**Set Up Hugging Face Hub and download model (Llama 2 Model)**



In [49]:
notebook_login()

In [50]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)


model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                              #load_in_8bit=True,
                                              load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
pipe=pipeline("text-generation",
              model=model,
              tokenizer=tokenizer,
              torch_dtype=torch.float16,
              device_map='auto',
              max_new_tokens=512,
              min_new_tokens=-1,
              top_k=30
              )

# Set up converstaion retrieval chain in Langchain

In [52]:
llm=HuggingFacePipeline(pipeline=pipe, model_kwargs={'temperature':0})

In [53]:
memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)

In [54]:
#Create our Q/A Chain
pdf_qa=ConversationalRetrievalChain.from_llm(llm=llm,
                                             retriever=docsearch.as_retriever(),
                                             verbose=False, memory=memory)

# Ask some questions based on our document

In [55]:
result=pdf_qa({"question":"what is Tivdak used for"})

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


In [56]:
result['answer']

' TIVDAK is a prescription medicine used to treat adults with cervical cancer that has returned or spread to other parts of the body, and who have received chemotherapy that did not work or is no longer working. It is not known if TIVDAK is safe and effective in children.\n\nPlease let me know if you have any other questions.'

In [57]:
result=pdf_qa({"question":"what are the risks of prescribing Tivdak?"})

In [58]:
result['answer']

'  Tivdak can cause severe and reversible ocular toxicity, including changes in vision, corneal ulceration, and eye pain. The risk of ocular toxicity is higher in patients with a history of previous eye disease or surgery. It is important to closely monitor patients for ocular toxicity and to withhold, reduce the dose, or permanently discontinue Tivdak if severe vision loss occurs. Additionally, Tivdak can cause immune-mediated adverse reactions, including hypersensitivity reactions, and drug interactions. It is important to carefully evaluate patients for immune-mediated adverse reactions and drug interactions before prescribing Tivdak.'